In [11]:
import pandas as pd
import numpy as np
!pip install xgboost
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

data = pd.read_csv('data.csv')
data.replace('M', 1,inplace=True)
data.replace('B',0, inplace=True)
data.drop(columns=['Unnamed: 32','id'],inplace=True)
data

In [ ]:
from dataclasses import dataclass
def normalization(df):
  # normalization calculation is  X - Xmin / Xmax - Xmin'
  normalized_df = df.copy()
  for name, values in normalized_df.iteritems():
    column_name = str(name)
    normalized_df[column_name] = (normalized_df[column_name] - normalized_df[column_name].min()) / (normalized_df[column_name].max() - normalized_df[column_name].min())

  return normalized_df

data = normalization(data)
data['diagnosis'] = data['diagnosis'].astype(int)
data

In [4]:
split = int(np.floor(0.8 * len(data)))
data = data.sample(frac=1,random_state=200)
x_train = data[:split]
y_train = data['diagnosis'][:split].values
x_test = data[split:]
y_test = data['diagnosis'][split:].values
print(split)
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

455
(455, 31) (455,) (114, 31) (114,)


In [12]:
# using grid search to evaluate: GridSearchCV
def hyperparameter_tuning(x_train, y_train):
  '''
    Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'sub_sample': 0.6}
    Lowest RMSE:  3.211133090353982e-05
  '''

  params = {
      'max_depth': [3, 5, 7, 9],
      'n_estimators': [100, 250, 500, 750, 1000],
      'learning_rate': [.01, 0.05, 0.1, 0.2, 0.3],
      'colsample_bytree': [0.5, 0.7, 1.0],
      'sub_sample': [0.6, 0.7, 0.8, 0.9, 1.0]}

  xgbr = xgb.XGBRegressor(seed = 20)

  clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=1)
  
  clf.fit(x_train, y_train)

  print("Best parameters:", clf.best_params_)
  print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))


In [13]:
hyperparameter_tuning(x_train, y_train)

Streaming output truncated to the last 5000 lines.
[18:45:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:45:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:45:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:45:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:45:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:45:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:45:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:45:59] WARNING: /workspace/src/objective/regression_obj.